# Checkpointing in FEniCSx

## Jørgen Schartum Dokken

### Simula Research Laboratory

FEniCS 23'

<div>
<p style="text-align:center;"><img src="fenics_banner.png" width="500">
</div>


# What is checkpointing?

> Checkpointing refers to the ability to store the state of a computation in a way that allows it be continued at a later time without changing the computation’s behavior [1]

In [19]:
import dolfinx
from mpi4py import MPI
import numpy as np
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 5))
u = dolfinx.fem.Function(V)
z = dolfinx.fem.Function(V)
steps_per_checkpoint = 5
for i, t in enumerate(np.linspace(0, 10, 16, endpoint=True)):
    u.interpolate(lambda x: t*x[0]+x[1])
    # Use u in a computation to update z
    # ....
    if i % steps_per_checkpoint == 0:
        # Call checkpointing function on z
        # ....
        print(f"Write checkpoint to file at {t=:.2f}")

Write checkpoint to file at t=0.00
Write checkpoint to file at t=3.33
Write checkpoint to file at t=6.67
Write checkpoint to file at t=10.00


[1] Schulz, M. (2011). Checkpointing. In: Padua, D. (eds) Encyclopedia of Parallel Computing. Springer, Boston, MA. [10.1007/978-0-387-09766-4_62](https://doi.org/10.1007/978-0-387-09766-4_62)

# What is the optimal format?

## FEM software
- Most finite element software use their own format for inputting/outputting meshes and functions
- No general consensus on what format to use

## Post-processing
- Most formats are based on [VTK](https://vtk.org/) in some shape or form
- Many different file formats support `vtk`
  - `.pvd`-files (`xml`-based)
  - `.xdmf`-files (`xml`+binary (`.h5`)
  - `.bp`-files (binary files in folder)

In [2]:
print("ss")

NameError: name 'ss' is not defined

In [6]:
print(55)

55
